# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [1]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [2]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"  #財經新聞

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [10]:
browser = webdriver.Chrome('./chromedriver')
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

SCROLL_PAUSE_TIME = 1

#
# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
#
last_height = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
while True:
    # Scroll down to the bottom.
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load the page.
    time.sleep(2)

    # Calculate new scroll height and compare with last scroll height.
    new_height = browser.execute_script("return document.body.scrollHeight")
    
    if new_height == last_height:
        break

    last_height = new_height
    '''
    Your code here
    
    提示：可參考以下的Stack Overflow: 
    https://stackoverflow.com/questions/48850974/selenium-scroll-to-end-of-page-indynamically-loading-webpage/48851166
    '''

### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [8]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [11]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)


----------------------------------------------------------------------

[0] 百萬用戶有福了！「玉山Wallet」APP再升級　3大功能超吸睛

▲「玉山Wallet」App再升級，推出3大進階版功能。（圖／玉山銀行提供）
記者紀佳妘／台北報導
「玉山Wallet」App去年推出國內首創的「即時綁卡」與「卡片管理」功能，玉山銀行今（28）日宣布，推出進階版「卡片管理2.0」、「消費類別查詢」與「一掃就Go！」三大功能，持續提供行動支付的便利服務。
玉山銀行指出，「卡片管理2.0」將深受民眾喜愛的「卡片管理」功能，加入單筆指定額度設定，提供多種不同金額的選項，讓顧客依每張卡片的使用習慣進行選擇，更容易管理每一張卡片，例如做為小額消費使用的信用卡，可設定單筆1,000元的限額，降低卡片遺失盜刷風險，讓卡友用卡更安心。
另在「消費類別查詢」功能，玉山信用卡的開發團隊運用數據分析與使用者回饋，了解顧客喜歡即時查詢消費狀況的需求，將消費分成一般消費、線上一般消費與超商條碼付共3種類型，透過分類功能讓顧客輕鬆記帳，打開「玉山Wallet」簡單的一個步驟，就能夠一目了然。
至於「一掃就Go！」功能，玉山銀行表示，將付款條碼整合載具、全家會員及會員折扣，透過「玉山Wallet」在全家消費時，僅需刷一次付款條碼，即可同時完成支付、發票存載具、累點並獲得會員優惠價，無須掃描多次條碼，未來也將繼續擴大至其他合作商店。
根據玉山銀行統計，「玉山Wallet」App下載量持續增加，至今已累計了超過百萬的下載量，為感謝卡友們長期支持，將於今年7月29日至8月17日舉辦「百萬用戶登入享千萬紅利」活動，活動期間「玉山Wallet」用戶登入達10天即可均分千萬紅利，還可再抽最高50,000點紅利點數。
此外，正值國內旅遊旺季，玉山信用卡要陪卡友走出台灣本島，到澎湖一同享受島嶼度假的風情，享受在地小吃及購買特產使用「玉山Wallet」掃碼付，掃描店家QR Code就能快速付款，今年8月1日至9月30日於指定店家消費，可享5%的現金回饋。
----------------------------------------------------------------------

[1]  美中新冷戰》沈榮欽／台股大漲肇因「新冷戰

▲台股今天早盤攻上13000點，再度改寫30年來的新高。（圖／實習記者方敬攝）
記者林淑慧／台北報導
台積電美國存託憑證（ADR）再飆漲逾12%，帶動台積電早盤跳空大漲再創466.5元新高價，推升台股早盤攻上13000點再寫歷史新頁，指數大漲441點。
美股27日三大指數同步收紅，道瓊工業指數上漲0.43%。標準普爾指數漲升0.74%，收在3239.41點。科技股那斯達克指數漲1.67%，收10536.27點。其中，台積電ADR飆漲逾12%，收83.25美元，股價再創歷史新高；聯電ADR更勁揚15.87%。
英特爾擬轉向委外代工，市場看好台積電可望受惠。台積電昨日攻上漲停，台積電ADR（美國存託憑證）周一再飆漲逾12%。今日開在464元，上漲9.31%；早盤飆上漲停466.5元。
其餘美股掛牌台股ADR周一皆表現亮眼，日月光控股ADR上漲5.97%，聯電ADR則大漲逾15%。聯電近來接單暢旺，昨日與台積電同步走強，並改寫逾12年來新高價，聯電今日開盤漲停。
鴻海今日開在77.8元，下跌0.26%；大立光（3008）以3835元開出，上漲0.52%。
----------------------------------------------------------------------

[12] 快訊／台股早盤飆漲逾400點　指數突破13000點大關

記者王曉敏／台北報導
台股今日以12951點開出，上漲363點；早盤持續上漲超過400點，正式突破13000點關卡。截至上午9點15分，台股指數暫報13010點，上漲422點，持續改寫歷史新高。
▲台股加權指數早盤一度突破13000點大關。（圖／實習記者方敬攝）
----------------------------------------------------------------------

[13] 台積電早盤飆漲停！台股指數突破13000點大關　持續改寫歷史新高

記者王曉敏／台北報導
在國際熱錢簇擁下，台股昨（27）日盤中一度觸及12686點，突破30年來高點。台股今日以12951點開出，上漲363點；早盤持續上漲逾400點，指數突破13000點大關，持續改寫歷史新高。
▲台積電早盤帶領台股早盤攻上13000點。（圖／實習記者方敬攝）
英特爾擬轉向委外代工，市場看好台積電可望受惠。台積電（233

記者柯沛辰／台北報導
威力彩頭獎連槓47期，原先預估上看31億元，但由於獎金飆高、買氣超乎預期，台彩27日下午臨時修改預估銷售額，預料頭獎將上看31.7億元，若有幸運兒一注獨得，扣稅後可實拿25.36億元，將是今年最高獎金，也將刷新史上新高紀錄。
▲威力彩頭獎飆上31.7億元，一注獨得將是史上最高獎金。（圖／記者黃國霖攝，下同）
威力彩上期47連槓，再度站上台彩2007年接手公益彩券發行以來最長連摃紀錄，也是第二次頭獎突破30億元大關。若有幸運兒一注獨得，扣掉2成所得稅後，還可得到25.36億元。
台彩27日下午緊急修改預估金額，將本期銷售額由18億增加至21億元，根據稍早開獎公布銷售金額為19.9億元，料將打破威力彩史上單期銷售金額紀錄，連帶使頭獎上看金額由31億元上修至31.7億元。
►資生堂晚安凍膜超殺650元！厚敷睡一覺就變美
----------------------------------------------------------------------

[25] 影／台積電可望躋身全球10大上市公司！分析師教你台股破新高的選股術

記者謝仁傑／綜合報導
台股今（27）日在台積電（2330）漲停的帶領下，攻破30年來的歷史高點，站上12,686點；各系外資紛紛上修台積電的目標價，500元已經達成共識。在今日《雲端最有錢》節目中，由分析師容逸燊、丁彥鈞、林睿閎來告訴大家，台股突破歷史高點之後，應如何操作股票。
英特爾7奈米製程延遲6個月，上週五下跌16.24％，反觀台積電客戶超微（AMD）大漲16.5％，台積電ADR漲了9.53％，促成台積電今日一舉攻上漲停價424.5元，市值有機會挑戰4,000億美元，躋身全球前10大上市公司。
台積電的漲停帶動晶圓代工股價跟近，第二大廠聯電也攻上漲停21.7元，改寫12年新高，世界先進漲幅也逾6％；此外，聯發科市值衝1.11兆，股價來到733元。新台幣也成為市場最強亞洲貨幣，飆升1.51角，最高來到29.405元，而黃金飆破1,923美元，再創歷史新高。
今年底與明年初，三大遊戲機 PS5、Xbox 和 Switch 會接連推出，遊戲新機上市的熱度和期待度很高，對其供應鏈必會產生話題性，像是創惟（6104）、南電（8046），不斷回檔後，周線緩步向上，鈺太首度打入遊戲機供應鏈，展望營收上拉，茂達（6138）、

▲鄧彥敦在紅火案中獲判無罪確定，求刑事補償60萬元。（圖／記者劉昌松攝）
記者吳銘峯／台北報導
前中信金法務長鄧彥敦，在「紅火案」中最終獲得無罪確定。他在本案中遭到羈押120日，因此無罪確定後，他向高等法院聲請刑事補償。鄧彥敦批評當年起訴的檢察官，訊問前就已先將故事劇本寫好，還不斷懷疑他的供詞，讓他無奈被羈押。因此請求刑是補償最高金額一日5000元，總計聲請60萬元。
「紅火案」發生於2004年，當時前總統陳水扁推動二次金改，中信金打算入股兆豐金。時任中信金副董事長的辜仲諒，指示公司前財務長張明田、前法務長鄧彥敦、前財務副總林祥曦等人，由子公司中信銀香港分行發行5億美元的次順位債券，其中3.9億美元買入股權型結構債，並利用建立結構債避險部位之名義，2005年底到2006年初，陸續買進兆豐金股票約44萬張，佔已發行股份總數3.97％，中信金順利入主兆豐金。
中信金入主兆豐金完成後，向金管會申報中信金及子公司當時所投資持有兆豐金控之股票數，辜仲諒在常務董事會未批示下，逕行將這批結構債由海外紙上公司Red Fire（紅火公司）買走。張明田等人再由紅火公司贖回公司債，操縱兆豐金控股價，從中獲利新台幣10億元。當時特偵組對辜仲諒、鄧彥敦等人依違反《證券交易法》間接操縱股價等罪及《銀行法》背信罪，提起公訴。
鄧彥敦一審遭判7年2月、二審加重為7年6月，但更一審獲判無罪。上訴最高法院後，最高法院去年11月駁回檢方上訴，鄧彥敦無罪確定。至於辜仲諒等人涉案部分，則撤銷發回高等法院進行更二審中。
鄧彥敦因此聲請刑事補償，他表示，案發時他年薪1600萬元、兒子僅12歲，目前兒子已經長大成人；而他現在的年薪更高達3000萬元。而他還批評當時起訴的檢察官，訊問前就已先將故事劇本寫好，還不斷懷疑他的供詞，更當庭罵他「強辯」，讓他無奈遭到羈押，纏訟14年才獲判無罪。所以他要求最高金額的補償，1日5000元，總計要求補償60萬元。法院審理後，諭知候核辦。
----------------------------------------------------------------------

[38] 賣破萬顆！超商銅板價「現烤可頌」開賣就排隊　預計20家→展店百家

▲全家賣現烤可頌，開賣首日就引發排隊潮，不到3個月銷售破萬顆。（圖／記者蕭筠攝）
記者林育綾／台北報導
超商搶攻烘焙商機！全家

記者陳心怡／台北報導
受到台積電大漲影響，國民ETF「元大台灣50 」0050今天漲4.8%，收盤價來到103.3元，盤中為103.55元，7月27日收盤與盤中價雙雙創歷史新高，不論盤中或收盤皆為目前市場唯一達三位數的ETF。
▲0050收盤與盤中價雙雙創歷史新高。（圖／達志圖庫）
法人分析，0050與台股大盤連動性高，不論自台股基本面、國際資金面來分析，台股長線仍具支撐力，0050也將連帶受惠，且表現可望優於大盤，且0050精選台股之優質龍頭企業，更具領漲抗跌特性。
專家表示，全球資金充沛，國際資金都欲尋找適合的投資標的，台股兼具「高殖利率」與「高修復力」之雙高優勢，將受資金追捧，建議投資人可選擇符合「高殖利率」、「優質」之龍頭標竿企業。
0050擁息收，同時兼具未來成長性，專家建議空手者欲投資可把握兩關鍵：「買黑不買紅」與「定期定額」。
台灣首檔ETF「元大台灣50 ETF」（0050）成立於2003年6月25日，不僅是現今規模最大、最具代表性的ETF，更是唯一一檔價格突破百元的ETF，0050受益人數6月底達17萬人，今年以來增83.39%。7月24日規模達1131.83億元，今年以來增70.92%。
----------------------------------------------------------------------

[46] 影／外資再喊「台積電將攻上530元」！分析師：這些供應鏈廠商也跟著受惠

記者陳心怡／台北報導
台積電發威，在台積電ADR上週五大漲近10%帶動下，今開盤股價飆漲34.5元，以420.5元開出，隨後持續攻高，終場以424.5元漲停板作收，同時市值也攀至11兆元，穩居台股市值第一，大盤指數甚至也衝破歷史高點12682點，而外資也全面看好台積電後市，同時調升目標價至530元。
由於英特爾7奈米量產延遲，並表態將尋找第三方夥伴外包，激勵台積電美ADR上週五大漲近10%，收在 73.9 美元。摩根士丹利、摩根大通均看好英特爾委外台積電代工機率大增，並可望加速台積電先進製程成長動能更進一步，各大外資接連發表樂觀看法，「500元等級」推測合理股價共識正在凝聚。摩根士丹利即預估，若英特爾委外台積電利多實現，台積電股價可望挑戰 530 元。
另外，摩根大通則調升台積電股價至 500 元，追平里昂；麥格理證券也調升台積電目標股價

▲透過記帳的方式，就可以更了解自己的花費。（圖／取自Pixabay）
記者曾筠淇／綜合報導
Elaine理白小姐整理了4種記帳技巧：
記帳除了可以了解自己的生活模式，也可以免於一些不必要的開銷，因此，Elaine理白小姐強調，「記帳讓你跑在正確的理財軌道上」，用這4種小技巧，讓記帳更有意義。
----------------------------------------------------------------------

[56] 全球七十五國抗疫評比「台灣名列第一」　助攻台上半年外銷訂單正成長

▲新任經濟部長王美花今(27)日就其管轄業務召開記者會，向媒體說明各項經濟政策推動進度 。（圖／記者湯興漢攝）
記者林淑慧／台北報導
新冠肺炎疫情衝擊全球，經濟部長王美花今（27）日表示，台灣防堵新冠肺炎疫情掌控得好，在外媒對全球七十五個經濟體的評比中，台灣抗疫表現名列第一，助攻我上半年外銷訂單仍維持正成長，而台商回流、五加二產業及六大核心產業等，經濟部也將逐步落實既定目標。
依據彭博評比75個新興和前沿經濟體對抗2019新冠疫情的表現，台灣整體表現居冠，第2到第5名依序是波札那、韓國、泰國和中國大陸，相較於亞洲部分國家快速復原，名列前茅，拉丁美洲國家則是吊車尾。
經濟部長王美花今天舉行上任後首次媒體茶敘，有關國內產業發展、國內外情勢發展變化之快，中美之間緊張關係對台灣的影響，台灣產業如何因應等議題，備受外界關注。
針對經濟部推動產業升級的進程，王美花指出，五加二產業的主軸仍將持續，蔡英文總統提到的六大核心產業中，過去針對醫療防疫物資的準備較有著墨，而台灣半導體的供應鏈做得愈完整，就能帶動高階製造，希望提升為全球最重要的半導體基地，讓國外高階的設備廠、材料廠能來台投資。
王美花指出，台商回台已有一兆五百多億元金額，業者持續落實投資，不諱言有些業者因應國際變化，回台投資腳步不如去年積極，經濟部仍會加緊協調。
另外，行政院已通過2,100億元的「紓困3.0」預算案，其中，經濟部編列1,376億元最多，交通部編列98億元。上述紓困案均執行至今年第4季，認定標準會在紓困2.0的基礎上微調。
王美花說明，主因紓困業務廣泛、三倍券金額龐大，水電紓困已超支等因素。她說，第3季的「紓困3.0」適用產業以製造業、會展產業為主，排除商業服務業，但納入進出口貿易服務業。
 
---

▲台灣微軟DevDays Asia 2020 Online。（圖／台灣微軟提供）
記者邱倢芯／台北報導
由台灣微軟主辦的「DevDays Asia 2020 Online亞太技術年會」系列活動即將於8月19日至8月26日接力登場，議程包含主題論壇、專家演講、數位工作坊、線上黑客松，以及產業應用分享等。
今年因應疫後新常態趨勢，活動不僅改為雲端交流，也匯聚全亞太區的IT領域專家、開發人員、學生，或是技術愛好者報名參加，推動台灣成為亞太人才培育中心，讓台灣全面Skill Up！
今年的「DevDays Asia 2020 Online亞太技術年會」也將圍繞七大主軸進行交流：
----------------------------------------------------------------------

[65] 影／台積電市值突破11兆！「預言家」張忠謀早點出台積電的未來命運

記者謝仁傑／綜合報導
英特爾在7奈米製程卡關後，台積電（2330）挾著對手利空創造利多優勢，今（27）日股價在盤中就攻達424.5的漲停價，市值突破11兆元大關，股價、市值雙雙創下新高，領軍台股寫下30年來的歷史新高點12,686點。台積電創辦人張忠謀曾在退休前最後一次股東會中預言，「台積電還會一次次再創奇蹟」。
張忠謀在2018年主持的最後一次股東會中，曾信心滿滿地說過，台積電將會持續進步，且會在他退休後一次次再創奇蹟。若按他當時持股12.5萬張台積電計算，在沒有賣掉的情況下，帳面價值已達到530億元，等於退休兩年身價翻了將近一倍。
去年11月張忠謀回到台積電參加運動會，當時正值劇烈的美中貿易戰，不過台積電仍保持逆勢成長之勢，作為全球供應鏈重要的一環，也許「預言家」張忠謀對於這個結果一點都不意外。
自從台股落實漲跌幅10％為限以來，這是台積電股價首度亮出紅燈，攻上漲停，應驗張忠謀當時「台積電將屢創奇蹟」的神預言。
在川普的華為禁令下，台積電今年5月宣布將於2021年赴美設廠，預計2024年開始用以量產5奈米晶片；隨著兩大巨頭三星和英特爾，都被台積電的先進製程拋在後頭，台積電也得以順利與主要客戶接單。或許張忠謀早已看到今日這種局勢，才選擇安心退休。
張忠謀期勉所有員工，當世界局勢不再安寧，台積電將會成為地緣政治家的必爭之地。他表示，台積電必須持續秉持三大優勢，創造技術領先、製造卓越

▲美國駐成都領事館關閉。（圖／路透）
●李沃牆／淡江大學財金系教授
美國於7月21日無預警要求中國關閉駐休士頓總領事館，指控該領事館是中方在美國境內竊取研究成果的中心，包括可能和新冠肺炎疫苗開發有關的科學研究。川普並於22日表示，「關閉更多領事館的可能性一直存在」。
其中，陸上海及深圳兩市三大指數於24日開低走低，上證指數終場收跌3.86%、深圳成指跌5.31%、創業板跌6.14%、港股恒生指數終場下跌557.67點，跌幅2.21%。另離岸及在岸人民幣原本在7月22 日就因美中關係緊張升級而走弱，雙雙跌破 7.0 關卡，在岸匯率來到7.0002，24日續貶至7.0162。川普劍指中方，其動機為何？地緣政治風險對金融市場影響甚鉅，陸股及人民幣未來走勢備受矚目。
▲川普選舉民調落後，外交手段也相對更趨激烈。（圖／路透）
川普民調節節敗退「抗中保川」奮力一博
美11月初的總統大選在即，川普選情隨疫情及經濟愈來愈嚴峻而快速下墜，據多家民調結果，川普落後競爭對手拜登（Joe Biden）達8至15個百分點。就連經濟學人（Economist）　的最新預測也指出，川普不僅在搖擺州（密西根州、威斯康辛州、賓州、佛羅里達州、亞利桑那州及北卡羅來納州）節節敗退，更危及共和黨票倉。
陸股短期高點修正　後勢有望震盪走高
大陸最早爆發疫情，也很快反映在股市，3月23日，上證指數來到2,660.17低點，但隨疫情控制得宜及經濟逐步解封、人行維持貨幣寬鬆政策，加上國際熱錢蜂擁而至、散戶歸隊而展開V型反彈；單是7月初短短幾個交易日的漲幅即逾10%，9日更漲至3,450.79點，創 2 年半來新高，期間波段的反彈幅度達29.7%。
據Wind統計，上半年滬深兩市股票累計成交額達到283.84兆元人民幣。然近期因第二波疫情有山雨欲來之勢、美中爭端再起，以及高點獲利調節賣壓之故，遂進入高檔震盪。但中長期而言，甫公布的第二季經濟成長由第一季的負6.8%轉為正3.2%，下半年基本面續佳及資金面寬鬆，第三及第四季更有望朝5%邁進，陸股應可獲支撐而繼續震盪走高。
▲人民幣的全球交易金額占比全球第五。（圖／pixabay）
人民幣短期將顛簸　中長期利多可維穩
人民幣國際化進程自2009年啟動以來已有相當不錯的進展，除2016年正式加入IMF特別提款權（SDR）一籃子貨幣外；根據環球銀行金融電信協會（SWIFT）於

▲三倍券無法找零，也不能用於儲值，支付使用時有限制性。（圖／記者黃士原攝）
● 單驥／國立中央大學終身榮譽教授、APIAA院士
根據2020年5月9日《信傳媒》引述BBC的報導，此次新冠疫情十分嚴重，故各國都祭出許多紓困方案，各方案中，最亮眼的政策就是發現金。
依該報導中BBC所整理的資料顯示，若以「新台幣」計算的直接現金給付政策來作比較，則以香港每人3.8萬元最高，其次分別為澳門3.7萬元、美國3.5萬元、日本2.7萬元、新加坡1.2萬元、南韓1萬元。而就台灣來說，在扣去民眾支付的新台幣1千元後，實際且有一些限制用途的「次現金」給付（抱歉，不能稱之為新台幣）為2千元。
許多民眾與媒體反映，這樣的給付，不論是給付水準、勞民與擾民的給付方式、支付使用時的限制性（故只能稱之為「次現金」）等，都很難讓人滿意，為此，其後續效果如何，仍有待客觀地評估。
從許多相關的資料顯示，台灣的振興券與日本地方政府發行的購物券有相當的相似性。
以2008年的金融海嘯為例，在當時，各國政府都致力於振興經濟，同時也要刺激消費。在各國諸多的刺激消費政策中，日本政府在當時所提出的購物券（等同台灣過去俗稱的「消費券」）是當時令人矚目的重要政策。
當時的政策是，由日本中央政府於2009年3月4日通過「特别定額給付金」的給付，在此一補付辦法規定下，日本每位國民可領取1萬2千日元的「補助」，此外，日本65歲以上，或18歲以下的國民，另可加發8千日元的定額給付金，上述金額，原則上，先由各地方公所函送申請表格至各住户，由住戶填具個人基本資料、申領意願（可以選擇不領）、銀行或郵儲帳户後，回送地方公所，約2～4週後，即可由日本政府將上述補助金匯進申請人帳户內。
由於日本國內國民的儲蓄率很高，日本政府擔心上述金額雖為數不大，但仍可能被存在銀行中不去消費，而喪失其鼓勵消費的美意，此時，日本的各個地方政府就另外出手，發行購物券，以刺激消費。
在當時，以日本首善之區東京都為例，由東京都中央區公所發行消費券，辦法是，民眾可以1萬日元現金換領1萬1千日元購物券，最高一人可購買5册（每册有1萬1千日元購物券，券面每張500日元）。此外，也附上一册参加購物券活動之各種商店、百貨公司、餐廳、醫院、服務業（交通事業除外）。當然，上述消費券的發放，是由各地方政府主導，其中，又以東京都最具指標意義。值得注意的是，上述地方政府消費券的發放